In [60]:
import pandas as pd
dataset=pd.read_csv('../tratamento/dataset_tratado.csv')

dataset_ibge=pd.read_csv('../tratamento/dataset_ibge_tratado.csv')

In [61]:
#Removendo caracteres indesejados no Nome 
dataset['Nome'] = dataset['Nome'].replace({',': ''}, regex=True)
dataset['Nome'] = dataset['Nome'].replace({'-': ''}, regex=True)
dataset = dataset.replace({'  ': ' '}, regex=True)
dataset['Nome'] = dataset['Nome'].replace({"'":' '}, regex=True)



In [62]:
#Valida os CPFS

from validate_docbr import CPF
#Valida os CPFS 
cpf = CPF()
def validaCPF(CPF):
    
    if(cpf.validate(CPF)):
        cpfValido ="Sim"
    else:
        cpfValido ="Não"

        
    return cpfValido

dataset['Validade CPF']=dataset['cpf'].apply(validaCPF)


In [68]:
#Seleciona apenas os CPFs validos
dataset=dataset.loc[dataset['Validade CPF']=="Sim"]
dataset.drop(columns=['Validade CPF'],inplace=True)

In [86]:
insercao = pd.DataFrame({'Nome':'Teste de CPF',
'cpf':'131.066.317-33',
'Cargo':'dsf',
'Tipo Cargo':'DEV',
'Cidade':'Guandu'
})



ValueError: If using all scalar values, you must pass an index

In [82]:
insercao=pd.DataFrame(insercao)

ValueError: If using all scalar values, you must pass an index